In [1]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import autoreject
import re
from pyprep.find_noisy_channels import NoisyChannels
from autoreject import AutoReject
%matplotlib qt5

In [3]:
rutas_archivos=[]
Docs_carpeta= os.listdir("C:\\Users\\jhquiza\\Downloads\\EDF\\")
for q in range(len(Docs_carpeta)):
    name_doc=Docs_carpeta[q]
    if ".edf" in name_doc:
        r1= r"C:\\Users\\jhquiza\\Downloads\\EDF\\"
        r1 = re.sub(r"\s+$", "", r1, flags=re.UNICODE)
        r= r1+name_doc
        rutas_archivos.append(r)
print(rutas_archivos)

['C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21100_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21101_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21102_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21103_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21104_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21105_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21106_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21107_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21108_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21109_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21110_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21111_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21112_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21113_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\\EDF\\\\21114_IAT2.edf', 'C:\\\\Users\\\\jhquiza\\\\Downloads\\\

In [4]:
# Se seleccionan solamente los eventos de los bloques 3,4,6 y 7 correspondientes a estímulos cuyas respuestas fueron correctas
def extraerCorrectas(eventos):
    pos=[]
    correctas=[]
    i=0
    j=0
    for i in range(len(eventos)):
        if eventos[i][2] == 12 and eventos[i+1][2]==108:
            pos.append(i)
        elif eventos[i][2] == 13 and eventos[i+1][2]==109:
            pos.append(i)
        elif eventos[i][2] == 14 and eventos[i+1][2]==110:
            pos.append(i)
        elif eventos[i][2] == 15 and eventos[i+1][2]==111:
            pos.append(i)
        elif eventos[i][2] == 16 and eventos[i+1][2]==112:
            pos.append(i)
        elif eventos[i][2] == 17 and eventos[i+1][2]==113:
            pos.append(i)
        elif eventos[i][2] == 18 and eventos[i+1][2]==114:
            pos.append(i)
        elif eventos[i][2] == 19 and eventos[i+1][2]==115:
            pos.append(i)
        elif eventos[i][2] == 24 and eventos[i+1][2]==120:
            pos.append(i)
        elif eventos[i][2] == 25 and eventos[i+1][2]==121:
            pos.append(i)
        elif eventos[i][2] == 26 and eventos[i+1][2]==122:
            pos.append(i)
        elif eventos[i][2] == 27 and eventos[i+1][2]==123:
            pos.append(i)    
        elif eventos[i][2] == 28 and eventos[i+1][2]==124:
            pos.append(i)
        elif eventos[i][2] == 29 and eventos[i+1][2]==125:
            pos.append(i)
        elif eventos[i][2] == 30 and eventos[i+1][2]==126:
            pos.append(i)
        elif eventos[i][2] == 31 and eventos[i+1][2]==127:
            pos.append(i)
    for j in range(len(pos)):
        x=int(pos[j])
        y=eventos[x]
        correctas.append(y)
    return correctas

In [20]:
# Primero eliminación e interpolación de canales malos, luego referenciación REST, luego ICA, luego epoching, luego remoción épocas malas
def Preprocesamiento_REST_ICA_epoch(raw, montage, decim=8):
    # Montaje
    raw.set_montage(montage=montage, match_case=False)
    # Preparación de referenciación REST - modelo de cabeza
    raw.del_proj()
    sphere = mne.make_sphere_model('auto', 'auto', raw.info)
    src = mne.setup_volume_source_space(sphere=sphere, exclude=30., pos=5.)
    forward = mne.make_forward_solution(raw.info, trans=None, src=src, bem=sphere)
    # Eliminación de canales ruidosos
    raw = raw.filter(l_freq=1,h_freq=None)
    NC = NoisyChannels(raw)
    NC.find_all_bads(ransac=False)
    print(NC.get_bads())
    raw.info["bads"] = NC.get_bads()
    # Interpolación de canales ruidosos
    raw.interpolate_bads(reset_bads=True, verbose=False)
    # Referenciación REST
    raw.set_eeg_reference(ref_channels='REST', projection=False, ch_type='auto', forward=forward)
        
    # Cálculo de componentes independientes
    ica = mne.preprocessing.ICA(n_components=None, random_state=1, method='fastica')
    ica.fit(raw)
    # Detección de artefactos
    ica.detect_artifacts(raw)
    # Detección de componentes oculares
    eog_indices, ___ = ica.find_bads_eog(raw, ch_name='FPZ')
    print('Componentes oculares: ',eog_indices)
    # Eliminación de componentes
    ica.apply(raw,exclude=eog_indices)
    ica.apply(raw,exclude=ica.exclude)
        
    # Epoching, selección de épocas de interés y submuestreo
    events = mne.find_events(raw, stim_channel='Status')
    event_id = [12, 13, 14, 15, 16, 17, 18, 19, 24, 25, 26, 27, 28, 29, 30, 31]
    epochs= mne.Epochs(raw,events=events,event_id=event_id,tmin=-0.2,tmax=0.8,baseline=(None,0),picks=['eeg'],preload=True,decim=decim)
    
    # Reparación y eliminación de épocas malas
    ar = AutoReject(verbose = False, random_state = 1)
    ar.fit(epochs)
    epochs_clean = ar.transform(epochs)
    return epochs_clean

In [25]:
montage = mne.channels.make_standard_montage('biosemi64')
for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    raw = mne.io.read_raw_edf(ruta, preload=True)
    epochs_clean = Preprocesamiento_REST_ICA_epoch(raw=raw, montage=montage)
    epochs_clean.save(ruta[-14:-9]+'n_epo.fif', overwrite=True)

Extracting EDF parameters from C:\Users\jhquiza\Downloads\EDF\21100_IAT2.edf...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
FP1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, TP7, CP5, CP3, CP1, P1, P3, P5, P7, P9, PO7, PO3, O1, IZ, OZ, POZ, PZ, CPZ, FPZ, FP2, AF8, AF4, AFZ, FZ, F2, F4, F6, F8, FT8, FC6, FC4, FC2, FCZ, CZ, C2, C4, C6, T8, TP8, CP6, CP4, CP2, P2, P4, P6, P8, P10, PO8, PO4, O2
Creating raw.info structure...
Reading 0 ... 987135  =      0.000 ...   482.000 secs...
Fitted sphere radius:         95.0 mm
Origin head coordinates:      0.0 -0.0 40.1 mm
Origin device coordinates:    0.0 -0.0 40.1 mm

Equiv. model fitting -> RV = 0.00348816 %
mu1 = 0.944717    lambda1 = 0.13715
mu2 = 0.667488    lambda2 = 0.683746
mu3 = -0.270376    lambda3 = -0.0105261
Set up EEG sphere model with scalp radius    95.0 mm

Sphere                : origin at (0.0 -0.0 40.1) mm
             

<ipython-input-20-7da7969caffb>:36: RuntimeWarning: The measurement information indicates a low-pass frequency of 417 Hz. The decim=8 parameter will result in a sampling frequency of 256 Hz, which can cause aliasing artifacts.
  epochs= mne.Epochs(raw,events=events,event_id=event_id,tmin=-0.2,tmax=0.8,baseline=(None,0),picks=['eeg'],preload=True,decim=decim)
Computing thresholds ...: 100%|████████████████████████████████████████████████████████| 64/64 [00:27<00:00,  2.34it/s]


Repairing epochs:   0%|                                                                        | 0/149 [00:00<?, ?it/s]

Repairing epochs:  32%|███████████████████▌                                          | 47/149 [00:00<00:00, 467.65it/s]

Repairing epochs:  63%|███████████████████████████████████████                       | 94/149 [00:00<00:00, 458.86it/s]

Repairing epochs: 100%|█████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 459.73it/s]

n_interp:   0%|                  

Dropped 4 epochs: 16, 67, 97, 105


Extracting EDF parameters from C:\Users\jhquiza\Downloads\EDF\21101_IAT2.edf...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
FP1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, TP7, CP5, CP3, CP1, P1, P3, P5, P7, P9, PO7, PO3, O1, IZ, OZ, POZ, PZ, CPZ, FPZ, FP2, AF8, AF4, AFZ, FZ, F2, F4, F6, F8, FT8, FC6, FC4, FC2, FCZ, CZ, C2, C4, C6, T8, TP8, CP6, CP4, CP2, P2, P4, P6, P8, P10, PO8, PO4, O2
Creating raw.info structure...
Reading 0 ... 1251327  =      0.000 ...   611.000 secs...
Fitted sphere radius:         95.0 mm
Origin head coordinates:      0.0 -0.0 40.1 mm
Origin device coordinates:    0.0 -0.0 40.1 mm

Equiv. model fitting -> RV = 0.00348816 %
mu1 = 0.944717    lambda1 = 0.13715
mu2 = 0.667488    lambda2 = 0.683746
mu3 = -0.270376    lambda3 = -0.0105261
Set up EEG sphere model with scalp radius    95.0 mm

Sphere                : origin at (0.0 -0.0 40.1) mm
            

<ipython-input-20-7da7969caffb>:36: RuntimeWarning: The measurement information indicates a low-pass frequency of 417 Hz. The decim=8 parameter will result in a sampling frequency of 256 Hz, which can cause aliasing artifacts.
  epochs= mne.Epochs(raw,events=events,event_id=event_id,tmin=-0.2,tmax=0.8,baseline=(None,0),picks=['eeg'],preload=True,decim=decim)
Computing thresholds ...: 100%|████████████████████████████████████████████████████████| 64/64 [00:29<00:00,  2.14it/s]


Repairing epochs:   0%|                                                                        | 0/157 [00:00<?, ?it/s]

Repairing epochs:  25%|███████████████▊                                              | 40/157 [00:00<00:00, 395.98it/s]

Repairing epochs:  51%|███████████████████████████████▌                              | 80/157 [00:00<00:00, 393.84it/s]

Repairing epochs: 100%|█████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 392.56it/s]

n_interp:   0%|                  

...

<ipython-input-20-7da7969caffb>:36: RuntimeWarning: The measurement information indicates a low-pass frequency of 417 Hz. The decim=8 parameter will result in a sampling frequency of 256 Hz, which can cause aliasing artifacts.
  epochs= mne.Epochs(raw,events=events,event_id=event_id,tmin=-0.2,tmax=0.8,baseline=(None,0),picks=['eeg'],preload=True,decim=decim)
Computing thresholds ...: 100%|████████████████████████████████████████████████████████| 64/64 [00:29<00:00,  2.15it/s]


Repairing epochs:   0%|                                                                        | 0/151 [00:00<?, ?it/s]

Repairing epochs:  32%|███████████████████▋                                          | 48/151 [00:00<00:00, 474.65it/s]

Repairing epochs:  64%|███████████████████████████████████████▍                      | 96/151 [00:00<00:00, 474.01it/s]

Repairing epochs: 100%|█████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 483.67it/s]

n_interp:   0%|                  

Dropped 22 epochs: 24, 64, 65, 66, 67, 68, 73, 97, 98, 102, 103, 104, 111, 112, 116, 118, 119, 121, 122, 123, 133, 134


Extracting EDF parameters from C:\Users\jhquiza\Downloads\EDF\24101_IAT2.edf...
EDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
FP1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, T7, TP7, CP5, CP3, CP1, P1, P3, P5, P7, P9, PO7, PO3, O1, IZ, OZ, POZ, PZ, CPZ, FPZ, FP2, AF8, AF4, AFZ, FZ, F2, F4, F6, F8, FT8, FC6, FC4, FC2, FCZ, CZ, C2, C4, C6, T8, TP8, CP6, CP4, CP2, P2, P4, P6, P8, P10, PO8, PO4, O2
Creating raw.info structure...
Reading 0 ... 1214463  =      0.000 ...   593.000 secs...
Fitted sphere radius:         95.0 mm
Origin head coordinates:      0.0 -0.0 40.1 mm
Origin device coordinates:    0.0 -0.0 40.1 mm

Equiv. model fitting -> RV = 0.00348816 %
mu1 = 0.944717    lambda1 = 0.13715
mu2 = 0.667488    lambda2 = 0.683746
mu3 = -0.270376    lambda3 = -0.0105261
Set up EEG sphere model with scalp radius    95.0 mm

Sphere                : origin at (0.0 -0.0 40.1) mm
            

<ipython-input-20-7da7969caffb>:36: RuntimeWarning: The measurement information indicates a low-pass frequency of 417 Hz. The decim=8 parameter will result in a sampling frequency of 256 Hz, which can cause aliasing artifacts.
  epochs= mne.Epochs(raw,events=events,event_id=event_id,tmin=-0.2,tmax=0.8,baseline=(None,0),picks=['eeg'],preload=True,decim=decim)
Computing thresholds ...: 100%|████████████████████████████████████████████████████████| 64/64 [00:29<00:00,  2.17it/s]


Repairing epochs:   0%|                                                                        | 0/149 [00:00<?, ?it/s]

Repairing epochs:  27%|████████████████▋                                             | 40/149 [00:00<00:00, 396.35it/s]

Repairing epochs:  54%|█████████████████████████████████▎                            | 80/149 [00:00<00:00, 396.07it/s]

Repairing epochs: 100%|█████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 388.45it/s]

n_interp:   0%|                  

Dropped 13 epochs: 5, 11, 23, 25, 26, 27, 33, 49, 52, 84, 122, 123, 144
